In [1]:
import os
import leafmap
from samgeo import SamGeo, tms_to_geotiff
from datetime import datetime
import json

In [2]:
# Obtendo a data e hora atual
now = datetime.now()
formatted_date = now.strftime("%Y_%m_%d_%H_%M")


def extract_coordinates(geojson_file):
    with open(geojson_file, 'r') as file:
        data = json.load(file)

    point_coords = []
    for feature in data['features']:
        if feature['geometry']['type'] == 'Point':
            point_coords.append(feature['geometry']['coordinates'])

    return point_coords

### Modelo

In [3]:
# vit_b(91M), vit_l(308M), vit_h(636M)

sam = SamGeo(
    model_type="vit_b",
    automatic=False,
    sam_kwargs=None,
)

### Imagem a ser segmentada

In [4]:
# imagem

image = "dados_google_earth/2004_07_recorte.tif"

# pasta saída, arquivo saída

pasta_saida = f'saida/mascaras_por_ponto/{os.path.splitext(os.path.basename(image))[0]}'

os.makedirs(pasta_saida, exist_ok=True)

mascaras_geradas = f'{pasta_saida}/mascaras_{formatted_date}.tif'

In [5]:
sam.set_image(image)

In [6]:
# pontos e detecção

point_coords = extract_coordinates(
    "dados_drone/2023_12.geojson")

print(point_coords)

[[735311.953296, 7016394.137161], [735311.953296, 7016326.969348], [735315.519729, 7016260.39594], [735320.274972, 7016202.144208], [735330.379864, 7016141.514855], [735351.184054, 7016058.8925], [735361.883352, 7016022.633769], [735364.855379, 7016014.312093]]


In [7]:
sam.predict(point_coords, point_labels=1,
            point_crs="EPSG:31982", output=mascaras_geradas)

In [8]:
# m = leafmap.Map(center=point_coords[0], zoom=19, height="400px")
# m.add_basemap("SATELLITE")
# m.layers[-1].visible = False
#
# m.add_raster(image, layer_name="Image")
#
# m.add_raster(mascaras_geradas, layer_name="Mask1",
#              nodata=0, cmap="Blues", opacity=1)
#
# m